# Introducción

Ya sabemos cómo leer archivos con pandas y cómo realizar una exploración básica del contenido. En esta sesión vamos a aprender algunas técnicas más avanzadas de Exploración de Datos. También vamos a ver los principios de la Limpieza de Datos, que usamos para dejar nuestros conjuntos de datos listos para ser reestructurados, analizados y visualizados.

La Exploración y la Limpieza van totalmente de la mano. No puedes limpiar sin explorar primero, y gran parte de las técnicas de exploración que usamos tienen como objetivo justamente encontrar inconsistencias, errores, redundancias, etc, en nuestro conjunto de datos para poder deshacernos de ellas.

# Objetivos

1. Leer archivos en formato CSV.
1. Utilizar técnicas más avanzadas de exploración de datos.
1. Utilizar la exploración para encontrar inconsistencias, errores y redundancias.
1. Usar algunas técnicas básicas de limpieza de datos.

La sesión pasada aprendimos a leer archivos en formato JSON. El formato JSON, que es muy parecido a los diccionarios de Python, es sólo uno de los tantos formatos con los que nos vamos a topar.

Los CSVs pertenecen a una clase de formatos donde las columnas de nuestra tabla se delimitan usando lo que se llama un separador. CSV significa Comma-Separated Values y como bien imaginarás significa que se una una coma (,) para separar las columnas. 

En un archivo de texto donde cada fila de nuestra tabla tiene su propia línea y donde los valores de cada columna se delimitan usando una coma (,). Leer archivos .csv usando pandas es muy fácil. Lo único que tienes que hacer es lo siguiente:

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../Datasets/melbourne_housing-raw.csv', sep=',')

pandas tiene un muy conveniente método llamado read_csv que nos permite leer archivos .csv directamente. Ése mismo método también puede ayudarnos a leer otros formatos con columnas delimitadas por otros separadores. Por ejemplo, podemos leer .tsv ('tab-separated values'), que son archivos donde cada columna está delimitada por un tab (indentación). Sólo basta con llamar el método con el argumento sep=\t.

# Análisis Exploratorio de Datos

Muy bien, empecemos nuestra exploración con las técnicas que ya conocemos primero:

In [4]:
df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [5]:
df.shape

(19740, 21)

In [6]:
df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [8]:
df.dtypes


Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

Ok, algunas cosas importantes:

1. El dataset tiene 19740 filas y 21 columnas
1. Podemos observar que la información que tiene el dataset es básicamente cierta descripción de las propiedades, su locación y tipo y fecha de venta.
1. Hay varias columnas float64, rooms es int64, y el resto son object.
1. Tenemos algunos Nans en el dataset.

¿NaNs? ¿Que quiero decir con eso? Bueno, los NaNs son valores Not a Number que básicamente son valores nulos en nuestro dataset. Estos valores pueden causar muchos problemas, ya que son valores nulos en columnas numéricas pero no podemos realizar operaciones matemáticas con ellos (son valores Not a Number, así que las matemáticas están fuera de las posibilidades). En esta sesión vamos a aprender a lidiar con estos valores, pero antes tenemos que aprender algo llamado funciones vectorizadas, que nos ayudará mucho durante esta exploración.

# Aritmética con Series y Funciones vectorizadas

¿Recuerdas nuestras funciones map y filter? A esas funciones les pasábamos nosotros una función y una lista y nos regresaban una lista con los resultados de aplicarle la función a cada uno de los elementos en orden. Las funciones vectorizadas funcionan muy parecido, pero están optimizadas para funcionar con arreglos de pandas y de numpy (si quieres saber más sobre numpy lee esto). Si tomas un arreglo de pandas (es decir, una Serie) y le aplicas una función vectorizada, la función se aplica a todo el arreglo elemento por elemento y te regresa un arreglo del mismo tamaño con el resultado de la aplicación.

Aplicar funciones de manera vectorizada a una Serie de pandas es facílisimo, incluso más fácil que usar la fución map. Primero veamos que podemos realizar una operación aritmética con una Serie y la operación se aplicará automáticamente a todo el arreglo "elemento por elemento":

In [9]:
serie = pd.Series([1,2,3,4,5])
serie * 5

0     5
1    10
2    15
3    20
4    25
dtype: int64

In [10]:
serie * 10

0    10
1    20
2    30
3    40
4    50
dtype: int64

In [11]:
(serie + 100) / 2

0    50.5
1    51.0
2    51.5
3    52.0
4    52.5
dtype: float64

No tenemos que usar map, ni que declarar una función, ni que usar lambda. Basta con una simple operación matemática. pandas además está optimizado para funcionar de esta manera, así que la velocidad de aplicación es mucho mayor que la combinación de map con listas.

Otra manera de aplicar estas transformaciones es usando funciones vectorizadas. Esto lo podemos hacer usando algunas funciones de numpy, por ejemplo. Numpy es otra librería que es muy común entre los científicos de datos. Ofrece muchas herramientas para realizar cálculos numéricos a altas velocidades. No usaremos mucho esta librería en este módulo, pero es importante entender que se pueden utilizar funciones de numpy para aplicar funciones de manera vectorizada a Series de pandas. Para importar numpy hacemos lo siguente:

In [12]:
import numpy as np

In [13]:
np.power(serie,2)

0     1
1     4
2     9
3    16
4    25
dtype: int64

También podemos sacar la raíz cuadrada de nuestra Serie:

In [14]:
np.sqrt(serie)

0    1.000000
1    1.414214
2    1.732051
3    2.000000
4    2.236068
dtype: float64

# Agregaciones

Hay una variación de estas funciones vectorizadas llamadas agregaciones (o reducciones) que lo que hacen es tomar un arreglo, atravesarlo "elemento por elemento" y regresar un solo número que es un "resumen" del arreglo. Este "resumen" es justamente la agregación o reducción. Podemos aplicar estas funciones usando numpy o directamente desde una Serie o DataFrame de pandas. Para efectos prácticos, vamos a utilizar nosotros los métodos que vienen integrados directamente en pandas. Por ejemplo, podemos sumar todos los valores de una Serie de esta manera:

In [15]:
serie.sum()

15

O podemos contar el número de elementos en una Serie así:

In [16]:
serie.count()

5

Podemos obtener el valor más pequeño de la Serie:

In [17]:
serie.min()

1

O el valor más grande:

In [18]:
serie.max()

5

# Funciones vectorizadas y Agregaciones con DataFrames

Tanto las funciones vectorizadas como las agregaciones pueden ser aplicadas a DataFrames completos.

Vemos primero las agregaciones. al aplicar una agregación a un DataFrame, lo que obtenemos de regreso es el resultado de aplicar la función a cada una de las columnas (que al final de cuentas son Series, ¿lo recuerdas?). Por ejemplo, tenemos este DataFrame:

In [22]:
df2 = pd.DataFrame(
    {
        'precio': [34,54,223,78,56,12,34],
        'cantidad_en_stock':[3,6,10,2,5,45,2],
        'productos_vendidos':[3,45,23,76,24,6,2]
    }, index=['Pokemaster', 'Cegatron', 'Pikame Mucho', 'Lazarillo de Tormes', 'Stevie Wonder', 'Needle', 'El AyMeDuele']
)

df2

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34,3,3
Cegatron,54,6,45
Pikame Mucho,223,10,23
Lazarillo de Tormes,78,2,76
Stevie Wonder,56,5,24
Needle,12,45,6
El AyMeDuele,34,2,2


Mira qué pasa cuando le aplicamos la agregación sum:

In [23]:
df2.sum()

precio                491
cantidad_en_stock      73
productos_vendidos    179
dtype: int64

pandas toma cada una de las columnas, suma todos los valores dentro de cada columna y nos regresa el resultado de las sumas en una nueva Serie, donde el índice son los nombres de las columnas en el DataFrame y los valores son las sumas.

También funciona con las demás agregaciones:

In [24]:
df2.min()

precio                12
cantidad_en_stock      2
productos_vendidos     2
dtype: int64

In [25]:
df2.max()

precio                223
cantidad_en_stock      45
productos_vendidos     76
dtype: int64

Ahora veamos qué pasa cuando aplicamos funciones vectorizadas a nuestro DataFrame:

In [26]:
df2 * 100

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,3400,300,300
Cegatron,5400,600,4500
Pikame Mucho,22300,1000,2300
Lazarillo de Tormes,7800,200,7600
Stevie Wonder,5600,500,2400
Needle,1200,4500,600
El AyMeDuele,3400,200,200


La función (x * 100) fue aplicada a cada uno de los elementos del DataFrame y obtuvimos un nuevo DataFrame con los resultados, ¿ves?

Cualquier función vectorizada que le apliquemos al DataFrame va a tener el mismo efecto:

In [28]:
df2 / 100

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,0.34,0.03,0.03
Cegatron,0.54,0.06,0.45
Pikame Mucho,2.23,0.10,0.23
Lazarillo de Tormes,0.78,0.02,0.76
Stevie Wonder,0.56,0.05,0.24
Needle,0.12,0.45,0.06
El AyMeDuele,0.34,0.02,0.02


Ahora sí, estamos listos para aprender a lidiar con valores nulos (NaNs).

# NaNs
Como ya dijimos, los NaNs (Not a Number) son valores nulos en nuestro conjunto de datos. Son valores que por alguna razón no se encuentran en nuestro dataset. A la hora de coleccionar nuestros datos, al momento de transcribirlos o de almacenarlos, algo pasó que algunos de esos datos faltan en el dataset final. pandas está diseñado para lidiar con estos datos fácilmente.

Veamos un DataFrame con valores NaN (estamos usando el objeto de numpy np.nan para crear valores Nan):

In [2]:

datos= {
        'precio': [34,54,np.nan,np.nan,56,12,34],
        'cantidad_en_stock':[3,6,np.nan,np.nan,5,2,10],
        'productos_vendidos':[3,45,23,np.nan,24,6,23]
    }

df3 = pd.DataFrame(datos,index=['Pokemaster', 'Cegatron', 'Pikame Mucho', 'Lazarillo de Tormes', 'Stevie Wonder', 'Needle', 'El AyMeDuele'])

NameError: name 'np' is not defined

In [ ]:
df3

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Lazarillo de Tormes,NaN,NaN,NaN
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


Como puedes ver, a este DataFrame le faltan datos. Los valores NaN que contiene son valores nulos, y nos nos dicen nada útil, en realidad. Podríamos asumir cosas acerca de por qué faltan estos datos, pero no serían más que suposiciones.

Ahora, ¿qué pasa con estos datos nulos? Su presencia puede ser problemática para algunos de los análisis que queremos realizar.

En este caso nuestro dataset es muy pequeño y podemos visualizarlo todo de un solo vistazo. Pero vamos a imaginar que nuestro dataset ese mucho más grande y que necesitamos saber si hay NaNs y cuántos. Para lograr esto podemos usar una función vectorizada llamada isna. isna checa cada valor en nuestro DataFrame, lo transforma en True cuando el valor es igual a NaN y a False cuando no lo es:

In [ ]:
df3.isna()

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,False,False,False
Cegatron,False,False,False
Pikame Mucho,True,True,False
Lazarillo de Tormes,True,True,True
Stevie Wonder,False,False,False
Needle,False,False,False
El AyMeDuele,False,False,False


Ok, ¿y ahora qué hacemos con esto? Podemos usar la agregación sum para hacer un conteo de nuestros valores nulos. Si recuerdas, sum sumaba todos los valores de cada columna y regresaba el total por columna. Si sumas valores booleanos, los Trues cuentan como 1 y los False cuentan como 0. Esto significa que aplicando la función, obtendremos el total de valores nulos en cada columna:

In [ ]:
df3.isna().sum()

precio                2
cantidad_en_stock     2
productos_vendidos    1
dtype: int64

De esta manera podemos saber si hay columnas que tienen demasiados valores nulos como para ser utilizadas.

También podemos obtener el número de NaNs que hay en cada fila pasándole un argumento a sum. Podemos indicarle a sum el eje en el cual queremos realizar la operación. En el caso de pandas eje se refiere a la dimensión de la estructura de datos. A estos ejes se les llama axis. Vamos a entender mejor los ejes más adelante cuando veamos aritmética de Series; por el momento basta con saber que si le pasamos axis=1 a sum, nos regresa la suma de NaNs por ìndice:



In [ ]:
df3.isna().sum(axis=1)

Pokemaster             0
Cegatron               0
Pikame Mucho           2
Lazarillo de Tormes    3
Stevie Wonder          0
Needle                 0
El AyMeDuele           0
dtype: int64

In [ ]:
df3.isna().sum(axis=0)

precio                2
cantidad_en_stock     2
productos_vendidos    1
dtype: int64

La decisión de qué hacer con los NaNs depende mucho del contexto. Vamos a ver 3 cosas básicas que podemos realizar para limpiar estos datos indeseables:

1. Eliminar filas con NaNs
1. Eliminar columnas con NaNs
1. Llenar los NaNs con algún valor.

# Eliminar filas con `NaNs`

Usando el método dropna, la eliminación de filas con `NaNs` se vuelve muy sencillo. Sólo basta con llamar `dropna` y todas las filas que contienen `NaNs` son eliminadas:



In [38]:
df3

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Lazarillo de Tormes,NaN,NaN,NaN
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


In [39]:
df3.dropna()

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


Esta operación no modifica el `DataFrame` original, así que si queremos que el cambio persista tenemos que asignarlo a un nuevo DataFrame:

In [41]:
df_dropped = df3.dropna()
df_dropped

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


En caso de que queramos eliminar solamente las filas donde TODOS los valores sean `NaN`, podemos pasarle el argumento `how='all'`:

In [42]:
df_dropped = df3.dropna(how='all')
df_dropped

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


El valor default es how='any', que elimina las filas donde haya mínimo un NaN.

# Eliminar columnas con `NaNs`

Ahora, ¿qué pasa si quisiéramos eliminar `NaNs` por columna? Vamos a agregar una columna a nuestro `DataFrame` que contenga puros `NaNs`:

In [46]:
df3['descuento'] = np.nan

In [47]:
df3

,precio,cantidad_en_stock,productos_vendidos,descuento
Pokemaster,34.0,3.0,3.0,NaN
Cegatron,54.0,6.0,45.0,NaN
Pikame Mucho,NaN,NaN,23.0,NaN
Lazarillo de Tormes,NaN,NaN,NaN,NaN
Stevie Wonder,56.0,5.0,24.0,NaN
Needle,12.0,2.0,6.0,NaN
El AyMeDuele,34.0,10.0,23.0,NaN


Si te fijas, sólo bastó con llamar df['descuento'] = np.nan para conseguir una columna completa de NaNs. Esto tiene que ver con la aritmética de Series. Cuando asigno un solo valor a una Serie, automáticamente toda la Serie toma ese valor.

Para eliminar columnas donde haya NaNs, llamamos también dropna pero con el argumento axis=1:

In [54]:
df3.dropna(axis=1, how='all')

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Lazarillo de Tormes,NaN,NaN,NaN
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


# Llenar valores nulos con un valor

Digamos que tenemos ahora un `DataFrame` que se ve así:

In [55]:
df3

,precio,cantidad_en_stock,productos_vendidos,descuento
Pokemaster,34.0,3.0,3.0,NaN
Cegatron,54.0,6.0,45.0,NaN
Pikame Mucho,NaN,NaN,23.0,NaN
Lazarillo de Tormes,NaN,NaN,NaN,NaN
Stevie Wonder,56.0,5.0,24.0,NaN
Needle,12.0,2.0,6.0,NaN
El AyMeDuele,34.0,10.0,23.0,NaN


Nuestra primera acción debería de ser eliminar las filas y columnas donde todos los valores sean `NaN`, porque no nos sirven de nada:

In [101]:
df_dropped = df3.dropna(axis=0, how='all')
df_dropped

,precio,cantidad_en_stock,productos_vendidos,descuento
Pokemaster,34.0,3.0,3.0,NaN
Cegatron,54.0,6.0,45.0,NaN
Pikame Mucho,NaN,NaN,23.0,NaN
Stevie Wonder,56.0,5.0,24.0,NaN
Needle,12.0,2.0,6.0,NaN
El AyMeDuele,34.0,10.0,23.0,NaN


In [102]:
df_dropped = df3.dropna(axis=1, how='all')
df_dropped

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Lazarillo de Tormes,NaN,NaN,NaN
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


Ahora, ¿qué debemos hacer con valores nulos que nos quedan? Digamos que nuestro análisis más importante tiene que ver con la columna 'precio', entonces esa columna es muy importante que esté limpia. Pero digamos que nuestra 'productos_vendidos' no es tan importante. Tal vez si hay un `NaN` en productos vendidos podemos asumir que no hay ningún producto vendido hasta ahora. En ese caso, podríamos llenar el/los NaN de la columna 'productos_vendidos' con `0s`. Eso se hace con el método `fillna`:

In [103]:
df_dropped

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Lazarillo de Tormes,NaN,NaN,NaN
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


In [104]:
df_dropped['productos_vendidos'].fillna(0)

Pokemaster              3.0
Cegatron               45.0
Pikame Mucho           23.0
Lazarillo de Tormes     0.0
Stevie Wonder          24.0
Needle                  6.0
El AyMeDuele           23.0
Name: productos_vendidos, dtype: float64

Seleccionamos la columna donde queremos llenar los `NaNs` con `0` y llamamos el métodos fillna(0). En este caso sólo estamos obteniendo de regreso la columna rellenada. Para tenerla en nuestro `DataFrame` podemos reasignarla a la misma columna:

In [105]:
df_dropped['productos_vendidos_2'] =  df_dropped['productos_vendidos'].fillna(0)
df_dropped

,precio,cantidad_en_stock,productos_vendidos,productos_vendidos_2
Pokemaster,34.0,3.0,3.0,3.0
Cegatron,54.0,6.0,45.0,45.0
Pikame Mucho,NaN,NaN,23.0,23.0
Lazarillo de Tormes,NaN,NaN,NaN,0.0
Stevie Wonder,56.0,5.0,24.0,24.0
Needle,12.0,2.0,6.0,6.0
El AyMeDuele,34.0,10.0,23.0,23.0


In [106]:
df_dropped = df_dropped.drop(columns=['productos_vendidos'])
df_dropped 

,precio,cantidad_en_stock,productos_vendidos_2
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Lazarillo de Tormes,NaN,NaN,0.0
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


In [109]:
df_cleaned = df_dropped.rename(columns={'productos_vendidos_2':'productos_vendidos'})

In [110]:
df_cleaned

,precio,cantidad_en_stock,productos_vendidos
Pokemaster,34.0,3.0,3.0
Cegatron,54.0,6.0,45.0
Pikame Mucho,NaN,NaN,23.0
Lazarillo de Tormes,NaN,NaN,0.0
Stevie Wonder,56.0,5.0,24.0
Needle,12.0,2.0,6.0
El AyMeDuele,34.0,10.0,23.0


# Aplicación en nuestro dataset original

Vamos a ver cómo funciona esto en nuestro dataset que teníamos al principio:

In [111]:
df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [114]:
df.shape

(19740, 21)

Primero hacemos conteo de NaNs:

In [136]:
df.isna().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price             4344
Method               0
SellerG              0
Date                 0
Distance             8
Postcode             8
Bedroom2          4413
Bathroom          4413
Car               4413
Landsize          4796
BuildingArea     11123
YearBuilt        10389
CouncilArea       4444
Lattitude         4292
Longtitude        4292
Regionname           8
Propertycount        8
dtype: int64

Tenemos un total de filas de 19740, así que el hecho de que tengamos alrededor de 11000 valores NaNs en las columnas BuildingArea y YearBuilt no son una buena señal. Tal vez después cambiemos nuestra decisión, pero por el momento vamos a simplemente eliminarlas:

In [137]:
df_2 = df.drop(columns=['BuildingArea', 'YearBuilt'])
df_2.isna().sum()

Suburb              0
Address             0
Rooms               0
Type                0
Price            4344
Method              0
SellerG             0
Date                0
Distance            8
Postcode            8
Bedroom2         4413
Bathroom         4413
Car              4413
Landsize         4796
CouncilArea      4444
Lattitude        4292
Longtitude       4292
Regionname          8
Propertycount       8
dtype: int64

Ok, ahora tenemos que decidir qué vamos a hacer con el resto de los NaNs. Presiento que no es muy grave no tener valores en 'Regionname', ya que es poco probable que usemos esa columna para nuestro análisis. Por lo tanto voy a llenar los NaNs con el valor Unknown:

In [138]:
df_2['Regionname'] = df_2['Regionname'].fillna('Unknown')


In [139]:
df_2.isna().sum()

Suburb              0
Address             0
Rooms               0
Type                0
Price            4344
Method              0
SellerG             0
Date                0
Distance            8
Postcode            8
Bedroom2         4413
Bathroom         4413
Car              4413
Landsize         4796
CouncilArea      4444
Lattitude        4292
Longtitude       4292
Regionname          0
Propertycount       8
dtype: int64

El resto de las columnas voy a considerarlas esenciales, así que vamos a eliminar todas las filas donde todavía tengamos NaNs:

In [140]:
df_dropped = df_2.dropna(axis=0, how='any')
df_dropped.isna().sum()

Suburb           0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64

Y ahora vemos cuántas filas nos han quedado:

In [141]:
df_dropped.shape

(11646, 19)

Si estas filas que nos quedan son suficientes o no, eso sólo lo sabremos continuando con el proceso.

# Reindexando

Ahora, algo pasó con nuestro dataset después de eliminar los NaNs y es que nuestro índice ya no corresponde con el número de filas en nuestro dataset:

In [143]:
df_dropped.tail(10)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
19719,Werribee,12 Bridge St,2,h,678000.0,S,YPA,29/07/2017,14.7,3030.0,3.0,1.0,4.0,1985.0,Hume,-37.58442,144.75565,Western Metropolitan,16166.0
19720,Werribee,18 Conquest Dr,4,h,645000.0,PI,hockingstuart,29/07/2017,14.7,3030.0,3.0,2.0,2.0,1994.0,Hume,-37.57499,144.75329,Western Metropolitan,16166.0
19722,Werribee,13b Kingfisher Ct,3,u,394000.0,PI,hockingstuart,29/07/2017,14.7,3030.0,3.0,1.0,0.0,1950.0,Brimbank,-37.78871,144.81369,Western Metropolitan,16166.0
19723,Werribee,40 Tarneit Rd,3,h,597000.0,S,Ray,29/07/2017,14.7,3030.0,3.0,1.0,2.0,1980.0,Brimbank,-37.79135,144.79948,Western Metropolitan,16166.0
19726,Westmeadows,43 Bamford Av,4,h,821000.0,S,YPA,29/07/2017,16.5,3049.0,3.0,2.0,2.0,2004.0,Brimbank,-37.70525,144.75699,Northern Metropolitan,2474.0
19731,Whittlesea,30 Sherwin St,3,h,601000.0,S,Ray,29/07/2017,35.5,3757.0,3.0,2.0,2.0,1970.0,Manningham,-37.76311,145.10494,Northern Victoria,2170.0
19734,Williamstown,87 Pasco St,3,h,1285000.0,S,Jas,29/07/2017,6.8,3016.0,2.0,1.0,1.0,2010.0,Whittlesea,-37.68199,145.01744,Western Metropolitan,6380.0
19737,Yarraville,2 Adeney St,2,h,750000.0,SP,hockingstuart,29/07/2017,6.3,3013.0,3.0,2.0,2.0,1999.0,Darebin,-37.75948,144.99615,Western Metropolitan,6543.0
19738,Yarraville,54 Pentland Pde,6,h,2450000.0,VB,Village,29/07/2017,6.3,3013.0,3.0,2.0,1.0,2011.0,Hume,-37.70322,144.88236,Western Metropolitan,6543.0
19739,Yarraville,10/127 Somerville Rd,3,t,645000.0,SP,Jas,29/07/2017,6.3,3013.0,2.0,1.0,1.0,1980.0,Hume,-37.69815,144.88019,Western Metropolitan,6543.0


Esto sucede porque eliminamos filas pero las filas que se mantuvieron siguen teniendo el mismo índice que antes. Hay veces que eso es lo que queremos (cuando nuestro índice son, nombres, etiquetas, letras, etc), pero en este caso, nos convendría que nuestro índice coincidiera con la posición de la fila en el dataset. Podemos corregir esto usando el método reset_index de la siguiente manera:

In [144]:
df_dropped.reset_index()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,19731,Whittlesea,30 Sherwin St,3,h,601000.0,S,Ray,29/07/2017,35.5,3757.0,3.0,2.0,2.0,1970.0,Manningham,-37.76311,145.10494,Northern Victoria,2170.0
11642,19734,Williamstown,87 Pasco St,3,h,1285000.0,S,Jas,29/07/2017,6.8,3016.0,2.0,1.0,1.0,2010.0,Whittlesea,-37.68199,145.01744,Western Metropolitan,6380.0
11643,19737,Yarraville,2 Adeney St,2,h,750000.0,SP,hockingstuart,29/07/2017,6.3,3013.0,3.0,2.0,2.0,1999.0,Darebin,-37.75948,144.99615,Western Metropolitan,6543.0
11644,19738,Yarraville,54 Pentland Pde,6,h,2450000.0,VB,Village,29/07/2017,6.3,3013.0,3.0,2.0,1.0,2011.0,Hume,-37.70322,144.88236,Western Metropolitan,6543.0


Como ves, el índice ahora corresponde con el número de filas que tenemos. El único problema es que ahora tenemos una columna llamada índice que contiene los índices anteriores. Una vez más, hay veces que queremos eso (cuando la información contenida ahí era relevante y no queremos deshacernos de ella), pero en este caso, en realidad no nos interesa mantener esa columna. Para resetear el índice y eliminarlo al mismo tiempo, usamos reset_index(drop=True):

In [145]:
df_dropped.reset_index(drop=True)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,Whittlesea,30 Sherwin St,3,h,601000.0,S,Ray,29/07/2017,35.5,3757.0,3.0,2.0,2.0,1970.0,Manningham,-37.76311,145.10494,Northern Victoria,2170.0
11642,Williamstown,87 Pasco St,3,h,1285000.0,S,Jas,29/07/2017,6.8,3016.0,2.0,1.0,1.0,2010.0,Whittlesea,-37.68199,145.01744,Western Metropolitan,6380.0
11643,Yarraville,2 Adeney St,2,h,750000.0,SP,hockingstuart,29/07/2017,6.3,3013.0,3.0,2.0,2.0,1999.0,Darebin,-37.75948,144.99615,Western Metropolitan,6543.0
11644,Yarraville,54 Pentland Pde,6,h,2450000.0,VB,Village,29/07/2017,6.3,3013.0,3.0,2.0,1.0,2011.0,Hume,-37.70322,144.88236,Western Metropolitan,6543.0


# Renombrando columnas

Además del índice, otros identificadores que tenemos que nos interesa mantener siempre limpios y claros son los nombres de nuestras columnas. En el caso de nuestro dataset, los nombres no son suficientemente homogéneos. Tenemos cosas como Regionname (segunda palabra con minúscula) y otras como CouncilArea (segunda palabra con mayúscula). Además, hay errores ortográficos (lattitude, longtitude). Y creo también que prefiero que los nombres sigan la convención de nombramiento de Python (snake_case). Voy a renombrar mis columnas entonces para que sigan las mismas convenciones.

Normalmente renombramos columnas cuando los nombres:

1. No son lo suficientemente claros
1. No representan la información que contiene esa columna.
1. Tienen información basura.
1. Tienen errores ortográficos.
1. No siguen la convención que hemos decidido que deberían de tener.
1. Son demasiado largos o difíciles de escribir.

Cambiemos entonces nuestros nombres de columnas. Primero creamos un mapa de los nombres viejos a los nombres nuevos:

In [146]:
df_dropped.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname',
       'Propertycount'],
      dtype='object')

In [147]:
column_name_mapping = {
    'Suburb':'suburb', 
    'Address':'address', 
    'Rooms': 'rooms', 
    'Type': 'type', 
    'Price' : 'price', 
    'Method': 'method', 
    'SellerG':'seller_g',
    'Date':'date', 
    'Distance':'distance', 
    'Postcode': 'postcode', 
    'Bedroom2': 'bedrooms', 
    'Bathroom': 'bathroom', 
    'Car':'car',
    'Landsize': 'land_size',
    'CouncilArea': 'council_area',
    'Lattitude':'latitude',
    'Longtitude':'longiude', 
    'Regionname':'region_name',
    'Propertycount':'property_count'
}

Y ahora usamos el método rename para cambiar los nombres:

In [148]:
df_renamed = df_dropped.rename(columns=column_name_mapping)
df_renamed.head()

,suburb,address,rooms,type,price,method,seller_g,date,distance,postcode,bedrooms,bathroom,car,land_size,council_area,latitude,longiude,region_name,property_count
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


Eso es todo